In [ ]:
import message_ix
import ixmp as ix
from message_ix.utils import make_df
from itertools import product
from timeit import default_timer as timer
import pandas as pd
import numpy as np
import warnings
warnings.filterwarnings('ignore')

import matplotlib.pyplot as plt
fs = 24
plt.style.use('seaborn-ticks')
plt.rcParams['axes.labelsize'] = fs
plt.rcParams['xtick.labelsize'] = fs
plt.rcParams['ytick.labelsize'] = fs
plt.rcParams['xtick.direction'] = 'out'
plt.rcParams['ytick.direction'] = 'out'
plt.rcParams['axes.axisbelow'] = True


<IPython.core.display.Javascript object>

### Steps taken:
1. Read curtailment parameters derived from PyPSA-Eur
2. Load scenario in MESSAGEix-GLOBIOM
3. Technology linkage (adding SDES and LDES)
4. Add curtailment technologies and relations to the scenario
5. Solve scenario

### Step 1) fetch curtailment parameters

1.1 Read data and define dictionaries:

In [ ]:
# gamma coefficients (marginal curtailment rates)
path = "parameters/"
gamma_ij_wind = pd.read_csv(path + "gamma_ij_wind_primary.csv",index_col=0).dropna()
gamma_ij_solar = pd.read_csv(path + "gamma_ij_solar_primary.csv",index_col=0).dropna()

# beta coefficients (counter acting term from renewable integration support measures)
techs = ["LDES","SDES"] 
beta_tech_wind = {}
beta_tech_solar = {}
for tech in techs:
    beta_tech_wind[tech] = pd.read_csv(path + "beta_"+tech+"_wind.csv",index_col=0).dropna()
    beta_tech_solar[tech] = pd.read_csv(path + "beta_"+tech+"_wind.csv",index_col=0).dropna()

# renewable shares
wind_shares_pypsa = pd.read_csv(path + "wind_shares.csv",index_col=0).loc[gamma_ij_wind.index]
solar_shares_pypsa = pd.read_csv(path + "solar_shares.csv",index_col=0).loc[gamma_ij_solar.index]

# create a dictionary with all the data
gamma_dict = gamma_ij_solar.to_dict()["0"]
gamma_wind_dict = gamma_ij_wind.to_dict()["0"]
gamma_dict.update(gamma_wind_dict) # dictionary containing all gamma coefficients

beta_dict = {}
for tech in techs:
    beta_dict[tech] = beta_tech_wind[tech].to_dict()["0"]

renewable_penetration_dict = solar_shares_pypsa.to_dict()["0"]
wind_penetration = wind_shares_pypsa.to_dict()["0"]
renewable_penetration_dict.update(wind_penetration)

Initialize relations:

In [ ]:
# number of bins (determined from the PyPSA-Eur data):
bins = list(set(renewable_penetration_dict.values()))
bins.sort()

bins_dict = {}
curt_relation = {} # i = solar, j = wind
curt_relation_tech = {}
new_bins = {}
prefix = "vre" # prefix of relation name
for i in range(len(bins)): # wind index
    for j in range(len(bins)): # solar index
        
        w = "wind_curtailment_w" + str(i) + "s" + str(j) # index naming in the data achieved from PyPSA-Eur
        s = "solar_curtailment_s" + str(j) + "w" + str(i) # index naming in the data achieved from PyPSA-Eur

        wind_curt_name = "wind_curtailment" + str(i+1) # naming of wind curtailment technology in MESSAGEix-GLOBIOM
        solar_curt_name = "solar_curtailment" + str(j+1) # naming of solar curtailment technology in MESSAGEix-GLOBIOM
        # we shift the index naming by one, since the index in PyPSA-Eur starts with 0, but in MESSAGEix-GLOBIOM with 1

        if (w not in gamma_ij_wind.index) and (s not in gamma_ij_solar.index): 
            continue
        
        new_bins[prefix + "_curtailment_w" + str(i+1) + "s" + str(j+1)] = [wind_curt_name, solar_curt_name]
        
        curt_relation[prefix + "_curtailment_w" + str(i+1) + "s" + str(j+1)] = [{wind_curt_name:gamma_ij_wind.loc[w].item(),
                                                                                solar_curt_name:gamma_ij_solar.loc[s].item()}]
        
        if j == 0:
            curt_relation_tech["wind_curtailment_" + str(i+1)] = wind_curt_name
            bins_dict["wind_curtailment_" + str(i+1)] = bins[i]
        
        if i == 0:
            curt_relation_tech["solar_curtailment_" + str(j+1)] = solar_curt_name
            bins_dict["solar_curtailment_" + str(j+1)] = bins[j]

curt_relation_tech = {k: curt_relation_tech[k] for k in sorted(curt_relation_tech)}

# print first five entries of the dictionaries
print({k: curt_relation[k] for k in list(curt_relation)[:5]})
print({k: curt_relation_tech[k] for k in list(curt_relation_tech)[:5]})

{'vre_curtailment_w1s1': [{'wind_curtailment1': 0.0037371249171338, 'solar_curtailment1': 0.002290072586197}], 'vre_curtailment_w1s2': [{'wind_curtailment1': 0.0276494369460103, 'solar_curtailment2': 0.2527864681633947}], 'vre_curtailment_w1s3': [{'wind_curtailment1': 0.0097918369226165, 'solar_curtailment3': 0.4818535505889985}], 'vre_curtailment_w1s4': [{'wind_curtailment1': 0.0010403098686803, 'solar_curtailment4': 0.1559578472451435}], 'vre_curtailment_w1s5': [{'wind_curtailment1': -0.001500484208699, 'solar_curtailment5': 0.0701529611080714}]}
{'solar_curtailment_1': 'solar_curtailment1', 'solar_curtailment_2': 'solar_curtailment2', 'solar_curtailment_3': 'solar_curtailment3', 'solar_curtailment_4': 'solar_curtailment4', 'solar_curtailment_5': 'solar_curtailment5'}


### Step 2) load scenario in MESSAGEix-GLOBIOM

In [ ]:
# load ixmp database
mp = ix.Platform('local',jvmargs=["-Xmx8G"])
solve_scenario = True # if True, solving scenario at the end
regions = ["R11_WEU"]

# reference scenario: 
sc_ref = message_ix.Scenario(mp, model="MESSAGEix-GLOBIOM", scenario="ENGAGE_SSP2_EN_NPi2020_500",version=1)
model = "MESSAGEix-GLOBIOM"
scen = "ENGAGE_SSP2_EN_NPi2020_500"

# clone scenario and add modifications
sc = sc_ref.clone(model=model, scenario=scen, keep_solution=False)

sc.check_out()

years = sc.set('year')

### Step 3) Technology linkage

**LDES**
- duration (E/G_d): 179.6 hrs (average of all scenarios)
- Efficiency (round-trip): 0.48
- Technology cost: 
    - Charge link: 450 EUR/kW (+ 2% FOM)
    - Storage: 12 EUR/kWh (+ 2% FOM)
    - Discharge link: 500 EUR/kW (+ 2% FOM) 
    Aggregated cost: 1/efficiency * 450 EUR/kW + duration * 12 EUR/kWh + 500 EUR/kW = 3,593 EUR/kW
- Capacity factor: 20.5%
- Lifetime: 20 years

**SDES**
- duration (E/G_d): 6 hrs
- Efficiency (round-trip): 0.94
- Technology cost: 
    - Charge link (battery inverter/converter): 160 EUR/kW
    - Storage: 142 EUR/kWh
    - Discharge link: 0 EUR/kW
    Aggregated cost: 1/effficiency * 160 EUR/kW + duration * 142 EUR/kWh + 0 EUR/kW = 1,022 EUR/kW
- Capacity factor: 11.9%
- Lifetime: 25 years


In MESSAGEix-GLOBIOM, the storage tech needs to have the following attributes defined:
- output (par)
- inv_cost (par)
- technology (set)
- var_cost (par)
- technical_lifetime (par)
- capacity_factor (par)

Storage technology attributes from PyPSA-Eur:

In [ ]:
ldes_inv_cost = 3593 # EUR/kW
sdes_inv_cost = 1022 # EUR/kW

ldes_lifetime = 20 # years
sdes_lifetime = 25 # years

ldes_capacity_factor = 0.205
sdes_capacity_factor = 0.119

Define pandas dataframes with storage attributes:

In [ ]:
df_ldes_output = make_df('output',
                        node_loc = "R11_WEU",
                        technology = "LDES",
                        year_vtg = years,
                        year_act = years,
                        mode = "M1",
                        node_dest = 'R11_WEU',
                        commodity = 'exports',
                        level = "secondary",
                        time = "year",
                        time_dest = "year",
                        value = 1,
                        unit = "GWa",
                        )

df_ldes_CF = make_df('capacity_factor',
                        node_loc = "R11_WEU",
                        technology = "LDES",
                        year_vtg = years,
                        year_act = years,
                        time = "year",
                        value = ldes_capacity_factor,
                        unit = "%",
                        )

df_ldes_inv_cost = make_df('inv_cost',
                           node_loc="R11_WEU",
                           technology="LDES",
                           year_vtg=years,
                           value=ldes_inv_cost,
                           unit="USD/GWa",
                           )

df_ldes_lifetime = make_df('technical_lifetime',
                           node_loc="R11_WEU",
                           technology="LDES",
                           year_vtg=years,
                           value=ldes_lifetime,
                           unit="y")

# sc.idx_names('capacity_factor')

Add storage attributes:

In [ ]:
tech = "LDES"
sc.add_set("technology", tech)
sc.add_par('output',df_ldes_output)
sc.add_par("capacity_factor", df_ldes_CF)
sc.add_par('inv_cost',df_ldes_inv_cost)
sc.add_par('technical_lifetime',df_ldes_lifetime)

Repeat for SDES ...

### Step 4) Add curtailment technologies and relations to the scenario

In [ ]:
# Curtailment relations and technologies (for checking)
curtail_relations = [x for x in set(sc.set("relation")) if "curtailment" in x]
curtail_techs = [x for x in set(sc.set("technology")) if "curtailment" in x]

# Add wind/solar curtailment data for new bins (not present in the old data)
new_relations = [x for x in bins_dict.keys() if x not in set(sc.set("relation"))]
new_techs = [curt_relation_tech[x] for x in new_relations]

# Add new set elements to the scenario
sc.add_set("technology", new_techs) # here, new technologies cover the ones for "wind_curtailment_{}" and "solar_curtailment_{}".
sc.add_set("relation", new_relations) # here, new relations cover the ones for "wind_curtailment_{}" and "solar_curtailment_{}".

Base curtailment:

In [ ]:
# In this step, we calculate the base curtailment, i.e., we don't consider
# the role of any curtailment-reducing technologies (contributors) like storage etc.
parname = "relation_activity"
for rel, region in product(sorted(bins_dict.keys()), regions):
    # Load existing data (and use it later for configuring the new data)
    old = sc_ref.par(parname, {"node_loc": region, 
                               "relation": rel})

    # If there is no data for this bin, use the data of the previous bin
    if old.empty:
        rel_pre = ("").join(
            [rel.split("_curt")[0],
             "_curtailment_",
             str(int(rel.split("ment_")[1]) - 1)]
            )
        old = sc_ref.par(parname, {"node_loc": region, "relation": rel_pre})

        # Replace relation and technology names for this bin
        curtail_tec = curt_relation_tech[rel_pre]
        old["relation"] = rel
        old.loc[old["technology"] == curtail_tec, "technology"] = curt_relation_tech[rel]

    else:
        curtail_tec = curt_relation_tech[rel]
        
    # Generate theoretical curtailment bins (with contributor technologies, but
    # these will be removed at the end, see Step (5))
    new = old.copy()
    # Edit the % share of wind/solar bins (Notice (-) sign)
    new.loc[new["technology"] == "elec_t_d", "value"] = -bins_dict[rel]

    # Add theoretical curtailment to the scenario
    sc.add_par(parname, new)

    # Add an upper bound for the theoretical curtailment (needed for new bins)
    bound = new.drop_duplicates(["node_rel", "year_rel", "relation"]).copy()
    bound["value"] = 0
    sc.add_par("relation_upper", bound)

    # Update the data of "input" electricity for this curtailment technology
    inp = sc.par("input", {"node_loc": region, "technology": curtail_tec})
    inp["technology"] = curt_relation_tech[rel] # is needed in case a new bin is added
    inp["value"] = 1
    sc.add_par("input", inp)
print("- New theoretical curtailment relations configured.")


- New theoretical curtailment relations configured.


In [ ]:
curt_relation_tech

{'solar_curtailment_1': 'solar_curtailment1',
 'solar_curtailment_2': 'solar_curtailment2',
 'solar_curtailment_3': 'solar_curtailment3',
 'solar_curtailment_4': 'solar_curtailment4',
 'solar_curtailment_5': 'solar_curtailment5',
 'solar_curtailment_6': 'solar_curtailment6',
 'solar_curtailment_7': 'solar_curtailment7',
 'wind_curtailment_1': 'wind_curtailment1',
 'wind_curtailment_2': 'wind_curtailment2',
 'wind_curtailment_3': 'wind_curtailment3',
 'wind_curtailment_4': 'wind_curtailment4',
 'wind_curtailment_5': 'wind_curtailment5',
 'wind_curtailment_6': 'wind_curtailment6',
 'wind_curtailment_7': 'wind_curtailment7'}

Add new bins to the set of relations and respective technologies to technology:

In [ ]:
sc.add_set("relation", new_bins.keys())

for rel_new, region in product(sorted(new_bins.keys()), regions):
    # Relevant wind/solar relations
    relations = [x for x, y in curt_relation_tech.items() if y in new_bins[rel_new][0].keys()]

    # Load the old data of contributors and use the average of solar and wind
    # TODO: The contributions must come from PyPSA. At the moment, the old values are
    # used for illustration
    old = sc.par(parname, {"node_loc": region, "relation": relations})
    if old.empty:
        continue

    # Keep only contributor and curtailment technologies
    # This does not need VRE generation and electricity grid, as the curtailment
    # bins as % of the grid were calculated in step (3) and here we use them
    techs = [x for x in set(old["technology"]) if
             not any([y in x for y in ["wind_r", "solar_r", "elec_t_d"]])]
    new = old.loc[old["technology"].isin(techs)].copy()

    # Group by technology and take the average of contributors values for solar and
    # wind for this VRE bin (e.g., if storage has 0.2 for wind, 0.1 for solar,
    # average(0.2 , 0.1) = 0.15 will be used for this VRE bin)
    new = new.groupby(
        ["node_rel", "node_loc", "mode", "technology", "year_rel", "year_act"]
                      ).mean(numeric_only=True)
    new = new.assign(relation=rel_new, unit="GWa").reset_index()

    # Change the sign of curtailment techs to (+1) to equalize them with contributors
    new.loc[new["technology"].isin(new_bins[rel_new]), "value"] = 1

    # Add the new equation for the VRE curtailment bin to the scenario
    sc.add_par(parname, new)

    # Add an upper bound for the new relation
    bound = new.drop_duplicates(["node_rel", "year_rel", "relation"]).copy()
    bound["value"] = 0
    sc.add_par("relation_upper", bound)

    # If treating input electricity at the VRE level (combination of solar-wind bins)
    
    # Relevant VRE technology
    tech_new = ("_").join(rel_new.split("_")[:2]) + rel_new.split("_")[2]
    
    # Add new VRE technology to the scenario
    sc.add_set("technology", tech_new)
    
    # Add VRE technology to this curtailment relation
    vre = new.loc[new["technology"].isin(new_bins[rel_new])]
    
    # Change the sign of VRE curtailment value to (-)
    # VRE curtailment is equal to the unmet curtailment of wind and solar, i.e.:
    # wind_curtail + solar_curtail <= storage (all contributors) + VRE_curtail
    vre = vre.assign(technology=tech_new, value=-1)
    sc.add_par(parname, vre)

    # Add "input" electricity for this VRE technology
    # Load sample data and copy it for each bin
    # if include_vre_input_loss:
    inp = sc.par("input", {"node_loc": region,
                            "technology": "solar_curtailment1"})
    
    # Add values for curtailed electricity at each combination of wind and solar
    # TODO: These values should come from PyPSA
    
    inp = inp.assign(technology=tech_new, 
                        value=1) #input_electr[rel_new])
    
    sc.add_par("input", inp)

print("- New VRE curtailment relations configured.")


- New VRE curtailment relations configured.


In [ ]:
df_relation_activity = sc.par("relation_activity")
print(df_relation_activity.query("relation == 'vre_curtailment_w6s2'").query("year_act == 2050"))

Remove the contributor technologies from theoretical relations:

In [ ]:
# Since they are already added to new VRE relations, the contributor technologies are removed from the theoretical relations
for rel, region in product(sorted(phi.keys()), regions):
    # Load existing data
    df = sc.par(parname, {"node_loc": region, "relation": rel})
    contributors = [x for x in set(df["technology"]) if
                    not any([y in x for y in ["wind", "solar", "elec_t_d"]])]
    df = df.loc[df["technology"].isin(contributors)].copy()
    # Remove contributors from theoretical relations
    sc.remove_par(parname, df)

    # Removing input electricity from wind and solar
    if not keep_input_loss:
        inp = sc.par("input", {"node_loc": region, "technology": rel_tech[rel]})
        sc.remove_par("input", inp)

sc.commit("Curtailment updated")

Solving scenario "MESSAGEix-GLOBIOM__ENGAGE_SSP2_EN_NPi2020_500__v9", please wait...!


### Solve scenario

In [ ]:
if solve_scenario:
    case = sc.model + '__' + sc.scenario + '__v' + str(sc.version)
    print(f'Solving scenario "{case}", please wait...!')

    start = timer()
    sc.solve(model='MESSAGE', case=case)
    end = timer()
    print('Elapsed time for solving scenario:', int((end - start)/60),
          'min and', round((end - start) % 60, 2), 'sec.')
    sc.set_as_default()


In [ ]:
mp.close_db()